In [2]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import datetime
import random
import re
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
import requests
import re
import py2neo as neo
import pandas as pd
import numpy as np
import random

In [3]:
db=neo.Graph("neo4j+s://52d54f1d.databases.neo4j.io",
    auth=("neo4j","cUQmozgEst1IHWWMmbpPgPiRExkAYWk-qSZdKYcALwA"))

In [4]:
teams_20_21=[]
teams_19_20=[]
teams_18_19=[]
teams_17_18=[]
teams_16_17=[]
teams_15_16=[]
teams_14_15=[]
teams_13_14=[]

In [5]:
Data_20_21=[]
Data_19_20=[]
Data_18_19=[]
Data_17_18=[]
Data_16_17=[]
Data_15_16=[]
Data_14_15=[]
Data_13_14=[]

In [6]:
def get_teams(year):
    html = urlopen("https://www.worldfootball.net/attendance/eng-premier-league-"+year+"/1/")
    bs = BeautifulSoup(html, 'html.parser')
    teams=[]
    table=bs.find("table",{'class': 'standard_tabelle'})
    for team in table.findAll("a"):
        teams.append(team.get_text())
    teams=list(filter(None,teams)) 
    return teams

In [7]:
def get_data(year):
    html = urlopen("https://www.worldfootball.net/attendance/eng-premier-league-"+year+"/1/")
    bs = BeautifulSoup(html, 'html.parser')
    Data=[]
    table=bs.find("table",{'class': 'standard_tabelle'})
    for d in table.findAll("td",{'align': 'right'}):
        Data.append(d.get_text())
    Data=list(filter(None,Data)) 
    return Data


In [8]:
teams_20_21=get_teams("2020-2021")
teams_19_20=get_teams("2019-2020")
teams_18_19=get_teams("2018-2019")
teams_17_18=get_teams("2017-2018")
teams_16_17=get_teams("2016-2017")
teams_15_16=get_teams("2015-2016")
teams_14_15=get_teams("2014-2015")
teams_13_14=get_teams("2013-2014")

In [9]:
Data_20_21=get_data("2020-2021")
Data_19_20=get_data("2019-2020")
Data_18_19=get_data("2018-2019")
Data_17_18=get_data("2017-2018")
Data_16_17=get_data("2016-2017")
Data_15_16=get_data("2015-2016")
Data_14_15=get_data("2014-2015")
Data_13_14=get_data("2013-2014")

In [10]:
teams=list( set((teams_20_21)+
 (teams_19_20)+
 (teams_18_19)+
 (teams_17_18)+
 (teams_16_17)+
 (teams_15_16)+
 (teams_14_15)+
 (teams_13_14) ) )

In [11]:

def create_node(nodes,type):
    for node in nodes:
        db.create(neo.Node(type,name=node))
    


In [12]:
create_node(teams,"stadium")

In [13]:
node_matcher = neo.NodeMatcher(db)
nodes_stadium=node_matcher.match("stadium")
nodes_stadium=list(nodes_stadium)

In [14]:
def update_data(team_name,datas,year):
    for node in nodes_stadium:
        if node["name"] in team_name:
            i=team_name.index(node["name"])
            node[year+"-sum/average"]=datas[i*2]+" / "+datas[i*2+1]
            db.push(node)
         

In [15]:
update_data(teams_20_21,Data_20_21,"20/21")
update_data(teams_19_20,Data_19_20,"19/20")
update_data(teams_18_19,Data_18_19,"18/19")
update_data(teams_17_18,Data_17_18,"17/18")
update_data(teams_16_17,Data_16_17,"16/17")
update_data(teams_15_16,Data_15_16,"15/17")
update_data(teams_14_15,Data_14_15,"14/15")
update_data(teams_13_14,Data_13_14,"13/14")

In [29]:
data_20_21=[]
teams_20_21=[]
for node in nodes_stadium:
    if node["20/21-sum/average"] is not None:
        data_20_21.append(node["18/19-sum/average"])
        teams_20_21.append(node["name"])

data_20_21,teams_20_21          

(['1.006.668 / 52.983',
  '768.297 / 40.437',
  '971.297 / 51.121',
  '1.108.375 / 58.336',
  None,
  '578.085 / 30.426',
  '736.815 / 38.780',
  '463.051 / 24.371',
  '483.644 / 25.455',
  None,
  '1.030.111 / 54.216',
  '1.415.471 / 74.498',
  '390.148 / 20.534',
  '1.028.477 / 54.130',
  '605.161 / 31.851',
  None,
  '589.572 / 31.030',
  '1.138.072 / 59.899',
  '572.640 / 30.139',
  None],
 ['Liverpool FC',
  'Chelsea FC',
  'Newcastle United',
  'West Ham United',
  'Sheffield United',
  'Brighton & Hove Albion',
  'Everton FC',
  'Fulham FC',
  'Crystal Palace',
  'Aston Villa',
  'Tottenham Hotspur',
  'Manchester United',
  'Burnley FC',
  'Manchester City',
  'Leicester City',
  'West Bromwich Albion',
  'Wolverhampton Wanderers',
  'Arsenal FC',
  'Southampton FC',
  'Leeds United'])